In [1]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [6]:
df_str_rel = pd.read_csv('/content/hin_pilot.csv', usecols=[0, 1])
df_str_rel.head()

,PairID,Text
0,HIN-pilot-00001,"""जॉर्जियाई अधिकारियों ने बताया कि 8 अगस्त 2008..."
1,HIN-pilot-00002,"""पुलिस आरोपित से अवैध हथियार की खरीद-फरोख्त के..."
2,HIN-pilot-00003,"""कई बार पहले भी बच्ची को जान से मारने की धमकी ..."
3,HIN-pilot-00004,"""धुंध से परेशान हैं, हिमाचल की पहाडियों का रूख..."
4,HIN-pilot-00005,"""इस दौरान वह दो बार गर्भवती हुई और चार बार उसन..."


In [7]:
df_str_rel['Text'].values

array(['"जॉर्जियाई अधिकारियों ने बताया कि 8 अगस्त 2008 को पोटी के काला सागर बंदरगाह पर लगभग 1 हवाई हमला तो हुआ था।"\r\n"2008 में जॉर्जिया के लोगों ने बताया कि 8 अगस्त को पोटी नाम की जगह पर हवाई जहाज़ से हमला हुआ था. पोटी काला सागर के पास एक बंदरगाह है।"',
       '"पुलिस आरोपित से अवैध हथियार की खरीद-फरोख्त के बारे में पूछताछ कर ही है।"\r\n"पर हमें दीर्घावधि के दृष्टिकोण से सोचना होगा।"',
       '"कई बार पहले भी बच्ची को जान से मारने की धमकी दे चुका था।"\r\n"इसी बात के विवाद में वह मायके भी चली गई थी।"',
       '"धुंध से परेशान हैं, हिमाचल की पहाडियों का रूख करें"\r\n"बोस ने कहा कि हो सकता है कि कोई ऐसा कर रहा हो।"',
       '"इस दौरान वह दो बार गर्भवती हुई और चार बार उसने खुदकुशी की कोशिश भी की।"\r\n"साथ ही उसने पीडि़त को झांसा दिया कि उसका कार्ड ब्लॉक हो गया है।"',
       '"जॉर्जिया के एक अलगाववादी क्षेत्र में हवाई हमलों ने संघर्ष को तीव्र किया।"\r\n"हवाई हमलों के कारण जॉर्जिया के एक हिस्से में, जहां कुछ लोग देश के बाकी हिस्सों से अलग रहना चाहते हैं, लड़ाई और भी बदतर हो गई है।"',
     

In [8]:
# Creating a column "Split_Text" which is a list of two sentences.
# df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel['Split_Text'] = df_str_rel['Text'].str.split('" "').apply(lambda x: [sentence.strip('"') for sentence in x])
df_str_rel.head()
print(df_str_rel['Split_Text'].iloc[1][0])

पुलिस आरोपित से अवैध हथियार की खरीद-फरोख्त के बारे में पूछताछ कर ही है।"
"पर हमें दीर्घावधि के दृष्टिकोण से सोचना होगा।


In [18]:
split_sentences = []

for index, row in df_str_rel.iterrows():
    temp3 = []
    text = row['Text']
    first_quote_index = text.find('"')
    second_quote_index = text.find('"', first_quote_index + 1)
    third_quote_index = text.find('"', second_quote_index + 1)
    fourth_quote_index = text.find('"', third_quote_index + 1)

    temp1 = text[first_quote_index+1 : second_quote_index]
    temp2 = text[third_quote_index + 1:fourth_quote_index]
    temp3.append(temp1)
    temp3.append(temp2)
    split_sentences.append(temp3)

df_str_rel['Split_Text'] = split_sentences

print(df_str_rel['Split_Text'].iloc[1])

['पुलिस आरोपित से अवैध हथियार की खरीद-फरोख्त के बारे में पूछताछ कर ही है।', 'पर हमें दीर्घावधि के दृष्टिकोण से सोचना होगा।']


In [19]:
temp1

'सीआइए ने दोनों को कैनाल कॉलोनी पुलिस को सुपुर्द कर दिया।'

In [21]:
!pip install sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=057e4eb867fc11c5437f41103d35792c0ab3c38fe1b4703ab28f5cd6535502bb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [22]:
!pip install tqdm


In [23]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

result_df = pd.DataFrame(columns=['PairID', 'Pred_Score'])

for index, row in tqdm(df_str_rel.iterrows(), total=len(df_str_rel)):
    sentences = row['Split_Text']

    embeddings = model.encode(sentences, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])

    result_df = result_df.append({'PairID': row['PairID'], 'Pred_Score': cosine_scores[0][0]}, ignore_index=True)

result_df = result_df.sort_values(by='Pred_Score', ascending=False)

result_df.to_csv('pred_eng_b.csv', index=False)


  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-23-3bd9d34d262f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'PairID': row['PairID'], 'Pred_Score': cosine_scores[0][0]}, ignore_index=True)
  5%|▌         | 1/20 [00:01<00:27,  1.44s/it]<ipython-input-23-3bd9d34d262f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'PairID': row['PairID'], 'Pred_Score': cosine_scores[0][0]}, ignore_index=True)
 10%|█         | 2/20 [00:01<00:13,  1.37it/s]<ipython-input-23-3bd9d34d262f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'PairID': row['PairID'], 'Pred_Score': cosine_scores[0][0]}, ignore_index=True)
 15%|█▌        | 3/20 [00:0

In [24]:
result_df.Pred_Score.iloc[23]

IndexError: ignored

In [32]:
result_df['Pred_Score'] = result_df['Pred_Score'].apply(lambda x: x.item())


In [25]:
result_df.Pred_Score.iloc[18]

tensor(0.0597)

In [33]:
result_df = result_df.sort_values(by='Pred_Score', ascending=False)

# Save the result as pred_eng_b.csv
result_df.to_csv('pred_eng_b.csv', index=False)

In [34]:
result_df

,PairID,Pred_Score
12,HIN-pilot-00013,0.892784
14,HIN-pilot-00015,0.888005
8,HIN-pilot-00009,0.822451
0,HIN-pilot-00001,0.787103
5,HIN-pilot-00006,0.759901
15,HIN-pilot-00016,0.738361
19,HIN-pilot-00020,0.498082
17,HIN-pilot-00018,0.472600
7,HIN-pilot-00008,0.451757
2,HIN-pilot-00003,0.437243


In [31]:
df_str_rel['Split_Text'].iloc[8]

['यह सिर्फ आधुनिक चीन की एक विशेषता है, इस बारे में हम कुछ भी नहीं कर सकते।',
 'यह बस कुछ ऐसा है जो आजकल चीन में होता है, और इसे बदलने के लिए हम कुछ नहीं कर सकते।']